# Problem Statement

Rainfall Prediction Using Artificial Neural Network on Multi-City Weather Data 
(2024–2025):

To design, train, and evaluate an **Artificial Neural Network (ANN)** that predicts whether it will rain the next day using historical weather data. 

**Keywords** - Data preprocessing, feature engineering, and ANN architectures.

Dataset Link - [Download](https://www.kaggle.com/datasets/waqi786/usa-rainfall-prediction-dataset-2024-2025/data)



In [3]:
# import dependancies

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Reading dataset

dataset = pd.read_csv("usa_rain_prediction_dataset_2024_2025.csv")
dataset.head()

,Date,Location,Temperature,Humidity,Wind Speed,Precipitation,Cloud Cover,Pressure,Rain Tomorrow
0,2024-01-01,New York,87.524795,75.655455,28.379506,0.000000,69.617966,1026.030278,0
1,2024-01-02,New York,83.259325,28.712617,12.436433,0.526995,41.606048,995.962065,0
2,2024-01-03,New York,80.943050,64.740043,14.184831,0.916884,77.364763,980.796739,1
3,2024-01-04,New York,78.097552,59.738984,19.444029,0.094134,52.541196,979.012163,0
4,2024-01-05,New York,37.059963,34.766784,3.689661,1.361272,85.584000,1031.790859,0


In [9]:
# check what are different features

print(dataset.columns.to_list())

['Date', 'Location', 'Temperature', 'Humidity', 'Wind Speed', 'Precipitation', 'Cloud Cover', 'Pressure', 'Rain Tomorrow']


In [10]:
# Dataset shape

print(dataset.shape)

(73100, 9)


In [11]:
# Basic info

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73100 entries, 0 to 73099
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           73100 non-null  object 
 1   Location       73100 non-null  object 
 2   Temperature    73100 non-null  float64
 3   Humidity       73100 non-null  float64
 4   Wind Speed     73100 non-null  float64
 5   Precipitation  73100 non-null  float64
 6   Cloud Cover    73100 non-null  float64
 7   Pressure       73100 non-null  float64
 8   Rain Tomorrow  73100 non-null  int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 5.0+ MB


Dataset hasn't missing values and all freatures are numeric except Date and Location

In [12]:
# Again check missing values

dataset.isnull().sum()

Date             0
Location         0
Temperature      0
Humidity         0
Wind Speed       0
Precipitation    0
Cloud Cover      0
Pressure         0
Rain Tomorrow    0
dtype: int64